# Gaussian Process Regression for Uncertainty Quantification

This notebook provides a comprehensive introduction to **Gaussian Process (GP) regression** for uncertainty quantification. Gaussian Processes are a powerful non-parametric Bayesian approach that naturally provides predictive uncertainty estimates.

## What You'll Learn

1. **GP Fundamentals**: Understanding GPs as distributions over functions
2. **Kernel Functions**: RBF and Matern kernels for modeling different smoothness assumptions
3. **Hyperparameter Optimization**: Automatic tuning via maximum likelihood
4. **Uncertainty Quantification**: Mean predictions and confidence intervals
5. **Interpolation vs Extrapolation**: How GP uncertainty changes with distance from training data
6. **Coverage Analysis**: Validating prediction intervals
7. **Noise Models**: Handling homoskedastic vs heteroskedastic noise

## References

- Rasmussen & Williams (2006). *Gaussian Processes for Machine Learning*
- scikit-learn GP documentation: https://scikit-learn.org/stable/modules/gaussian_process.html

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys
sys.path.insert(0, '../..')

# Gaussian Process from sklearn
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import (
    RBF, Matern, WhiteKernel, ConstantKernel as C
)
from sklearn.metrics import mean_squared_error

# Dataset utilities
from src.datasets.nonlinear import GaussianDataset, ExponentialDecayDataset
from src.datasets.generators import generate_noise

# Visualization
from src.visualization import setup_plot_style

# Metrics
from src.metrics import picp, mean_interval_width

# Set up plotting style
setup_plot_style()
%matplotlib inline

# Set random seed for reproducibility
np.random.seed(42)

print("All libraries imported successfully!")

## 1. Gaussian Processes: The Basics

### What is a Gaussian Process?

A **Gaussian Process** is a collection of random variables, any finite number of which have a joint Gaussian distribution. In the context of regression:

- **Prior**: We assume that the unknown function $f(x)$ follows a GP prior: $f(x) \sim \mathcal{GP}(m(x), k(x, x'))$
  - $m(x)$ is the mean function (often set to 0)
  - $k(x, x')$ is the covariance (kernel) function that encodes our assumptions about smoothness

- **Posterior**: After observing training data $(X_{\text{train}}, y_{\text{train}})$, we get a posterior GP:
  $$f(x_*) | X_{\text{train}}, y_{\text{train}}, x_* \sim \mathcal{N}(\mu(x_*), \sigma^2(x_*))$$
  
  where:
  - $\mu(x_*)$ is the predictive mean
  - $\sigma^2(x_*)$ is the predictive variance

### Key Properties

1. **Non-parametric**: GPs don't assume a fixed functional form
2. **Bayesian**: Naturally provides uncertainty quantification
3. **Kernel-based**: The kernel function encodes prior knowledge about function smoothness
4. **Exact inference**: Closed-form posterior (for Gaussian likelihood)

In [ ]:
# Visualize GP prior samples
def plot_gp_prior_samples(kernel, n_samples=5, n_points=200):
    """
    Visualize samples from a GP prior.
    
    This shows what kind of functions the GP believes are plausible
    BEFORE seeing any data.
    """
    X_plot = np.linspace(0, 1, n_points).reshape(-1, 1)
    
    # Create GP with the kernel
    gp = GaussianProcessRegressor(kernel=kernel, random_state=42)
    
    # Sample from prior
    y_samples = gp.sample_y(X_plot, n_samples=n_samples)
    
    # Plot
    fig, ax = plt.subplots(figsize=(12, 5))
    for i in range(n_samples):
        ax.plot(X_plot, y_samples[:, i], alpha=0.7, linewidth=1.5, label=f'Sample {i+1}')
    
    ax.set_xlabel('x')
    ax.set_ylabel('f(x)')
    ax.set_title(f'GP Prior Samples\nKernel: {kernel}')
    ax.legend(loc='best', fontsize=9)
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# Try different kernels
print("RBF Kernel (Squared Exponential):")
print("Infinitely differentiable, very smooth functions")
kernel_rbf = C(1.0) * RBF(length_scale=0.1)
plot_gp_prior_samples(kernel_rbf)

In [ ]:
print("\nMatern Kernel (nu=1.5):")
print("Once differentiable, less smooth than RBF")
kernel_matern = C(1.0) * Matern(length_scale=0.1, nu=1.5)
plot_gp_prior_samples(kernel_matern)

## 2. Kernel Selection and Hyperparameters

### Common Kernels

**RBF (Radial Basis Function / Squared Exponential)**:
$$k(x, x') = \sigma^2 \exp\left(-\frac{\|x - x'\|^2}{2\ell^2}\right)$$
- $\sigma^2$: output variance (amplitude)
- $\ell$: length scale (controls smoothness)
- Produces infinitely smooth functions

**Matern Kernel**:
$$k(x, x') = \sigma^2 \frac{2^{1-\nu}}{\Gamma(\nu)} \left(\frac{\sqrt{2\nu}\|x - x'\|}{\ell}\right)^\nu K_\nu\left(\frac{\sqrt{2\nu}\|x - x'\|}{\ell}\right)$$
- $\nu$ controls smoothness ($\nu = 1/2, 3/2, 5/2$ common)
- Less smooth than RBF, often more realistic

**White Kernel (Noise)**:
$$k(x, x') = \sigma_n^2 \delta_{x,x'}$$
- Models observation noise
- Only non-zero when $x = x'$

### Hyperparameter Optimization

Hyperparameters (like length scale and variance) are automatically optimized by maximizing the **marginal likelihood**:
$$\log p(y | X, \theta) = -\frac{1}{2}y^T K^{-1} y - \frac{1}{2}\log|K| - \frac{n}{2}\log(2\pi)$$

In [ ]:
# Demonstrate hyperparameter optimization
def demonstrate_hyperparameter_optimization():
    """Show how GP optimizes kernel hyperparameters."""
    
    # Generate synthetic data
    np.random.seed(42)
    X_train = np.random.uniform(0.2, 0.8, 15).reshape(-1, 1)
    y_train = np.sin(10 * X_train).flatten() + 0.1 * np.random.randn(15)
    
    X_test = np.linspace(0, 1, 200).reshape(-1, 1)
    
    # Define kernel with bounds for hyperparameters
    kernel = C(1.0, (1e-3, 1e3)) * RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e2))
    
    print(f"Initial kernel: {kernel}")
    
    # Fit GP
    gp = GaussianProcessRegressor(
        kernel=kernel,
        n_restarts_optimizer=10,  # Try 10 random starting points
        random_state=42
    )
    gp.fit(X_train, y_train)
    
    print(f"\nOptimized kernel: {gp.kernel_}")
    print(f"Log-marginal-likelihood: {gp.log_marginal_likelihood(gp.kernel_.theta):.3f}")
    
    # Predictions
    y_pred, y_std = gp.predict(X_test, return_std=True)
    
    # Plot
    fig, ax = plt.subplots(figsize=(12, 5))
    
    # Training data
    ax.scatter(X_train, y_train, c='red', s=80, alpha=0.8, edgecolors='black', 
               linewidth=1.5, label='Training data', zorder=10)
    
    # GP mean and uncertainty
    ax.plot(X_test, y_pred, 'b-', linewidth=2, label='GP mean', zorder=5)
    ax.fill_between(X_test.flatten(), 
                     y_pred - 1.96*y_std, 
                     y_pred + 1.96*y_std,
                     alpha=0.3, color='blue', label='95% confidence', zorder=1)
    ax.fill_between(X_test.flatten(), 
                     y_pred - y_std, 
                     y_pred + y_std,
                     alpha=0.5, color='blue', label='68% confidence', zorder=2)
    
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_title('GP with Optimized Hyperparameters')
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

demonstrate_hyperparameter_optimization()

## 3. GP Regression on Nonlinear Data

Let's apply GP regression to a **Gaussian (bell curve)** function with noise. This is a common nonlinear pattern in scientific data.

In [ ]:
# Load Gaussian dataset
dataset = GaussianDataset(
    n_samples=100,
    noise_model='homoskedastic',
    noise_level=0.05,  # 5% noise
    seed=42,
    a=1.0,
    mu=0.5,
    sigma=0.15
)

# Generate data
data = dataset.generate()

print(f"Dataset: {dataset.get_function_form()}")
print(f"Training samples: {len(data.X_train)}")
print(f"Test samples: {len(data.X_test)}")
print(f"Gap samples: {len(data.X_gap)}")
print(f"Noise model: {dataset.noise_model}")
print(f"Noise level: {dataset.noise_level}")

In [ ]:
# Visualize the dataset
fig, ax = plt.subplots(figsize=(12, 6))

# Plot regions
ax.scatter(data.X_train, data.y_train, alpha=0.7, s=50, 
           label='Training data', color='blue', edgecolors='black', linewidth=0.5)
ax.scatter(data.X_gap, data.y_gap, alpha=0.7, s=50, 
           label='Gap (interpolation test)', color='orange', marker='s')
ax.scatter(data.X_extrap_low, data.y_extrap_low, alpha=0.7, s=50, 
           label='Low extrapolation', color='red', marker='^')
ax.scatter(data.X_extrap_high, data.y_extrap_high, alpha=0.7, s=50, 
           label='High extrapolation', color='green', marker='v')

# True function
X_true = np.linspace(0, 1, 300).reshape(-1, 1)
y_true = dataset._generate_clean(X_true.flatten())
ax.plot(X_true, y_true, 'k--', linewidth=2, label='True function', zorder=10)

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('Gaussian Function Dataset with Homoskedastic Noise')
ax.legend(loc='best', fontsize=9)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Fitting a Gaussian Process Model

We'll fit a GP with an **RBF kernel** and **White kernel** for noise.

In [ ]:
# Prepare data
X_train = data.X_train.reshape(-1, 1)
y_train = data.y_train
X_test = data.X_test.reshape(-1, 1)
y_test = data.y_test

# Combine test and gap for full evaluation
X_gap = data.X_gap.reshape(-1, 1) if data.X_gap is not None else np.array([]).reshape(-1, 1)
y_gap = data.y_gap if data.y_gap is not None else np.array([])

if len(X_gap) > 0:
    X_eval = np.vstack([X_test, X_gap])
    y_eval = np.hstack([y_test, y_gap])
else:
    X_eval = X_test
    y_eval = y_test

# Define kernel: ConstantKernel * RBF + WhiteKernel
kernel = (
    C(1.0, (1e-3, 1e3)) * RBF(length_scale=0.1, length_scale_bounds=(1e-3, 1.0)) +
    WhiteKernel(noise_level=0.01, noise_level_bounds=(1e-5, 1e1))
)

print("Initial kernel:")
print(kernel)

# Create and fit GP
gp = GaussianProcessRegressor(
    kernel=kernel,
    n_restarts_optimizer=10,
    alpha=1e-10,
    normalize_y=True,
    random_state=42
)

print("\nFitting Gaussian Process...")
gp.fit(X_train, y_train)

print("\nOptimized kernel:")
print(gp.kernel_)
print(f"\nLog-marginal-likelihood: {gp.log_marginal_likelihood(gp.kernel_.theta):.3f}")

In [ ]:
# Make predictions
y_pred, y_std = gp.predict(X_eval, return_std=True)

# Compute 95% prediction intervals
z_score = 1.96  # 95% confidence
y_lower = y_pred - z_score * y_std
y_upper = y_pred + z_score * y_std

# Compute metrics
rmse = np.sqrt(mean_squared_error(y_eval, y_pred))
coverage = np.mean((y_eval >= y_lower) & (y_eval <= y_upper))
mean_width = np.mean(y_upper - y_lower)

print("\n" + "="*50)
print("GP PERFORMANCE METRICS")
print("="*50)
print(f"RMSE:                {rmse:.4f}")
print(f"Coverage (95% PI):   {coverage:.3f} (target: 0.95)")
print(f"Mean Interval Width: {mean_width:.4f}")
print("="*50)

## 5. Uncertainty Visualization

Let's visualize the GP predictions with uncertainty bands.

In [ ]:
# Create dense grid for smooth plotting
X_plot = np.linspace(0, 1, 300).reshape(-1, 1)
y_pred_plot, y_std_plot = gp.predict(X_plot, return_std=True)
y_lower_plot = y_pred_plot - 1.96 * y_std_plot
y_upper_plot = y_pred_plot + 1.96 * y_std_plot

# True function
y_true_plot = dataset._generate_clean(X_plot.flatten())

fig, ax = plt.subplots(figsize=(14, 6))

# Uncertainty bands (plot first so they're in background)
ax.fill_between(X_plot.flatten(), y_lower_plot, y_upper_plot,
                 alpha=0.2, color='blue', label='95% Prediction Interval')
ax.fill_between(X_plot.flatten(), 
                 y_pred_plot - y_std_plot, 
                 y_pred_plot + y_std_plot,
                 alpha=0.4, color='blue', label='68% Prediction Interval')

# Predictions and true function
ax.plot(X_plot, y_pred_plot, 'b-', linewidth=2.5, label='GP Mean Prediction', zorder=5)
ax.plot(X_plot, y_true_plot, 'k--', linewidth=2, label='True Function', zorder=6)

# Training data
ax.scatter(X_train, y_train, c='red', s=70, alpha=0.8, 
           edgecolors='black', linewidth=1, label='Training Data', zorder=10)

# Highlight regions
ax.axvspan(0, 0.125, alpha=0.05, color='red', label='Low Extrapolation')
ax.axvspan(0.875, 1.0, alpha=0.05, color='green', label='High Extrapolation')

ax.set_xlabel('x', fontsize=12)
ax.set_ylabel('y', fontsize=12)
ax.set_title('Gaussian Process Regression with Uncertainty Quantification', fontsize=14, fontweight='bold')
ax.legend(loc='best', fontsize=9)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nKey Observations:")
print("1. Uncertainty (shaded region) is SMALL near training data")
print("2. Uncertainty INCREASES in extrapolation regions (x < 0.125 and x > 0.875)")
print("3. GP mean closely follows the true function in interpolation region")
print("4. In extrapolation regions, GP reverts to prior mean with high uncertainty")

## 6. Visualizing Uncertainty Growth in Extrapolation

One of the key features of GPs is that uncertainty **increases** as we move away from training data.

In [ ]:
# Plot standard deviation vs x
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# Top plot: Predictions with uncertainty
axes[0].fill_between(X_plot.flatten(), y_lower_plot, y_upper_plot,
                      alpha=0.3, color='blue')
axes[0].plot(X_plot, y_pred_plot, 'b-', linewidth=2, label='GP Mean')
axes[0].plot(X_plot, y_true_plot, 'k--', linewidth=2, label='True Function')
axes[0].scatter(X_train, y_train, c='red', s=50, alpha=0.8, 
                edgecolors='black', linewidth=1, label='Training Data', zorder=10)
axes[0].set_ylabel('y', fontsize=12)
axes[0].set_title('GP Predictions and 95% Confidence Bands', fontsize=12, fontweight='bold')
axes[0].legend(loc='best')
axes[0].grid(True, alpha=0.3)

# Bottom plot: Standard deviation
axes[1].plot(X_plot, y_std_plot, 'r-', linewidth=2.5, label='Predictive Std Dev')
axes[1].axvspan(0, 0.125, alpha=0.1, color='red', label='Low Extrap')
axes[1].axvspan(0.875, 1.0, alpha=0.1, color='green', label='High Extrap')
axes[1].set_xlabel('x', fontsize=12)
axes[1].set_ylabel('Standard Deviation', fontsize=12)
axes[1].set_title('Uncertainty (Standard Deviation) vs Position', fontsize=12, fontweight='bold')
axes[1].legend(loc='best')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nObservations:")
print("- Uncertainty is LOWEST near training data points")
print("- Uncertainty INCREASES dramatically in extrapolation regions")
print("- This is a key advantage of GPs: honest uncertainty quantification")

## 7. Effect of Different Kernels

Let's compare RBF vs Matern kernels on the same data.

In [ ]:
# Define different kernels
kernels = {
    'RBF': C(1.0, (1e-3, 1e3)) * RBF(0.1, (1e-3, 1.0)) + WhiteKernel(0.01, (1e-5, 1e1)),
    'Matern (nu=1.5)': C(1.0, (1e-3, 1e3)) * Matern(0.1, (1e-3, 1.0), nu=1.5) + WhiteKernel(0.01, (1e-5, 1e1)),
    'Matern (nu=2.5)': C(1.0, (1e-3, 1e3)) * Matern(0.1, (1e-3, 1.0), nu=2.5) + WhiteKernel(0.01, (1e-5, 1e1)),
}

# Fit GPs with different kernels
gp_results = {}

for name, kernel in kernels.items():
    print(f"\nFitting GP with {name} kernel...")
    gp_temp = GaussianProcessRegressor(
        kernel=kernel,
        n_restarts_optimizer=10,
        alpha=1e-10,
        normalize_y=True,
        random_state=42
    )
    gp_temp.fit(X_train, y_train)
    
    # Predict
    y_pred_temp, y_std_temp = gp_temp.predict(X_plot, return_std=True)
    
    # Evaluate
    y_pred_eval, y_std_eval = gp_temp.predict(X_eval, return_std=True)
    y_lower_eval = y_pred_eval - 1.96 * y_std_eval
    y_upper_eval = y_pred_eval + 1.96 * y_std_eval
    
    rmse_temp = np.sqrt(mean_squared_error(y_eval, y_pred_eval))
    coverage_temp = np.mean((y_eval >= y_lower_eval) & (y_eval <= y_upper_eval))
    width_temp = np.mean(y_upper_eval - y_lower_eval)
    
    gp_results[name] = {
        'gp': gp_temp,
        'y_pred': y_pred_temp,
        'y_std': y_std_temp,
        'rmse': rmse_temp,
        'coverage': coverage_temp,
        'width': width_temp
    }
    
    print(f"  Optimized kernel: {gp_temp.kernel_}")
    print(f"  RMSE: {rmse_temp:.4f}, Coverage: {coverage_temp:.3f}, Width: {width_temp:.4f}")

In [ ]:
# Plot comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for ax, (name, result) in zip(axes, gp_results.items()):
    y_pred_temp = result['y_pred']
    y_std_temp = result['y_std']
    
    # Uncertainty bands
    ax.fill_between(X_plot.flatten(), 
                     y_pred_temp - 1.96*y_std_temp, 
                     y_pred_temp + 1.96*y_std_temp,
                     alpha=0.3, color='blue', label='95% CI')
    
    # Predictions
    ax.plot(X_plot, y_pred_temp, 'b-', linewidth=2, label='GP Mean')
    ax.plot(X_plot, y_true_plot, 'k--', linewidth=1.5, label='True Function')
    
    # Training data
    ax.scatter(X_train, y_train, c='red', s=40, alpha=0.8, 
               edgecolors='black', linewidth=0.5, label='Training', zorder=10)
    
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_title(f'{name}\nCoverage: {result["coverage"]:.3f}, RMSE: {result["rmse"]:.4f}')
    ax.legend(loc='best', fontsize=8)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKernel Comparison:")
print("- RBF: Very smooth, infinite differentiability")
print("- Matern(1.5): Once differentiable, allows rougher functions")
print("- Matern(2.5): Twice differentiable, smoother than Matern(1.5)")

## 8. Coverage Analysis

Let's analyze prediction interval coverage across different regions.

In [ ]:
# Define regions
X_eval_flat = X_eval.flatten()

regions = {
    'Low Extrap': (X_eval_flat < 0.125),
    'Interpolation': (X_eval_flat >= 0.2) & (X_eval_flat <= 0.8),
    'Gap': (X_eval_flat >= 0.375) & (X_eval_flat <= 0.625),
    'High Extrap': (X_eval_flat > 0.875),
}

# Compute coverage by region for each kernel
coverage_by_region = {}

for name, result in gp_results.items():
    gp_temp = result['gp']
    y_pred_eval, y_std_eval = gp_temp.predict(X_eval, return_std=True)
    y_lower_eval = y_pred_eval - 1.96 * y_std_eval
    y_upper_eval = y_pred_eval + 1.96 * y_std_eval
    
    coverage_by_region[name] = {}
    for region_name, mask in regions.items():
        if np.sum(mask) > 0:
            y_region = y_eval[mask]
            y_lower_region = y_lower_eval[mask]
            y_upper_region = y_upper_eval[mask]
            coverage_region = np.mean((y_region >= y_lower_region) & (y_region <= y_upper_region))
            coverage_by_region[name][region_name] = coverage_region
        else:
            coverage_by_region[name][region_name] = np.nan

# Create DataFrame
coverage_df = pd.DataFrame(coverage_by_region).T
print("\nCoverage by Region (95% Prediction Intervals):")
print("="*60)
print(coverage_df.to_string())
print("="*60)
print("Target coverage: 0.95\n")

In [ ]:
# Visualize coverage by region
fig, ax = plt.subplots(figsize=(12, 6))

x_pos = np.arange(len(regions))
width = 0.25

for i, (name, coverages) in enumerate(coverage_by_region.items()):
    values = [coverages.get(region, 0) for region in regions.keys()]
    ax.bar(x_pos + i*width, values, width, label=name, alpha=0.8)

ax.axhline(y=0.95, color='red', linestyle='--', linewidth=2, label='Target (0.95)')
ax.set_xlabel('Region', fontsize=12)
ax.set_ylabel('Coverage', fontsize=12)
ax.set_title('Prediction Interval Coverage by Region and Kernel', fontsize=14, fontweight='bold')
ax.set_xticks(x_pos + width)
ax.set_xticklabels(regions.keys())
ax.legend(loc='best')
ax.grid(True, alpha=0.3, axis='y')
ax.set_ylim([0, 1.05])
plt.tight_layout()
plt.show()

## 9. Homoskedastic vs Heteroskedastic Noise

GPs can handle both constant noise (homoskedastic) and input-dependent noise (heteroskedastic).

In [ ]:
# Create datasets with different noise models
dataset_homo = GaussianDataset(
    n_samples=100,
    noise_model='homoskedastic',
    noise_level=0.05,
    seed=42
)

dataset_hetero = GaussianDataset(
    n_samples=100,
    noise_model='heteroskedastic',
    noise_level=0.10,
    seed=42
)

data_homo = dataset_homo.generate()
data_hetero = dataset_hetero.generate()

# Fit GPs
datasets_noise = {
    'Homoskedastic': data_homo,
    'Heteroskedastic': data_hetero
}

gp_noise_results = {}

for noise_type, data_temp in datasets_noise.items():
    X_train_temp = data_temp.X_train.reshape(-1, 1)
    y_train_temp = data_temp.y_train
    X_test_temp = data_temp.X_test.reshape(-1, 1)
    y_test_temp = data_temp.y_test
    
    # Define kernel
    kernel_temp = (
        C(1.0, (1e-3, 1e3)) * RBF(0.1, (1e-3, 1.0)) +
        WhiteKernel(0.01, (1e-5, 1e1))
    )
    
    # Fit GP
    gp_temp = GaussianProcessRegressor(
        kernel=kernel_temp,
        n_restarts_optimizer=10,
        alpha=1e-10,
        normalize_y=True,
        random_state=42
    )
    gp_temp.fit(X_train_temp, y_train_temp)
    
    # Predict
    y_pred_temp, y_std_temp = gp_temp.predict(X_plot, return_std=True)
    
    # Evaluate
    y_pred_test, y_std_test = gp_temp.predict(X_test_temp, return_std=True)
    y_lower_test = y_pred_test - 1.96 * y_std_test
    y_upper_test = y_pred_test + 1.96 * y_std_test
    
    coverage_temp = np.mean((y_test_temp >= y_lower_test) & (y_test_temp <= y_upper_test))
    
    gp_noise_results[noise_type] = {
        'data': data_temp,
        'gp': gp_temp,
        'y_pred': y_pred_temp,
        'y_std': y_std_temp,
        'coverage': coverage_temp
    }
    
    print(f"\n{noise_type} Noise:")
    print(f"  Optimized kernel: {gp_temp.kernel_}")
    print(f"  Coverage: {coverage_temp:.3f}")

In [ ]:
# Plot comparison
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

for ax, (noise_type, result) in zip(axes, gp_noise_results.items()):
    data_temp = result['data']
    y_pred_temp = result['y_pred']
    y_std_temp = result['y_std']
    
    # Uncertainty bands
    ax.fill_between(X_plot.flatten(), 
                     y_pred_temp - 1.96*y_std_temp, 
                     y_pred_temp + 1.96*y_std_temp,
                     alpha=0.3, color='blue', label='95% CI')
    
    # Predictions
    ax.plot(X_plot, y_pred_temp, 'b-', linewidth=2.5, label='GP Mean')
    ax.plot(X_plot, y_true_plot, 'k--', linewidth=2, label='True Function')
    
    # Training data
    ax.scatter(data_temp.X_train, data_temp.y_train, c='red', s=50, alpha=0.7, 
               edgecolors='black', linewidth=0.5, label='Training', zorder=10)
    
    ax.set_xlabel('x', fontsize=12)
    ax.set_ylabel('y', fontsize=12)
    ax.set_title(f'{noise_type} Noise\nCoverage: {result["coverage"]:.3f}', 
                 fontsize=12, fontweight='bold')
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nObservations:")
print("- Homoskedastic: Constant noise level, uniform uncertainty bands")
print("- Heteroskedastic: Input-dependent noise, but GP assumes constant noise")
print("- GP with WhiteKernel models homoskedastic noise by default")
print("- For heteroskedastic noise, consider using input-dependent noise models")

## 10. Calibration Quality: Expected vs Observed Coverage

A well-calibrated GP should have **observed coverage ≈ nominal coverage** across different confidence levels.

In [ ]:
# Compute coverage for different confidence levels
confidence_levels = np.linspace(0.5, 0.99, 20)
observed_coverages = []

for conf_level in confidence_levels:
    # Compute z-score for this confidence level
    from scipy.stats import norm
    z = norm.ppf((1 + conf_level) / 2)
    
    # Compute intervals
    y_pred_calib, y_std_calib = gp.predict(X_eval, return_std=True)
    y_lower_calib = y_pred_calib - z * y_std_calib
    y_upper_calib = y_pred_calib + z * y_std_calib
    
    # Compute coverage
    coverage_calib = np.mean((y_eval >= y_lower_calib) & (y_eval <= y_upper_calib))
    observed_coverages.append(coverage_calib)

observed_coverages = np.array(observed_coverages)

# Plot calibration curve
fig, ax = plt.subplots(figsize=(10, 8))

# Perfect calibration line
ax.plot([0.5, 1.0], [0.5, 1.0], 'k--', linewidth=2, label='Perfect Calibration', zorder=1)

# Observed calibration
ax.plot(confidence_levels, observed_coverages, 'bo-', linewidth=2.5, 
        markersize=8, label='GP Calibration', zorder=5)

# Shaded region for acceptable calibration
ax.fill_between(confidence_levels, confidence_levels - 0.05, confidence_levels + 0.05,
                 alpha=0.2, color='green', label='±5% tolerance')

ax.set_xlabel('Nominal Coverage (Confidence Level)', fontsize=12)
ax.set_ylabel('Observed Coverage', fontsize=12)
ax.set_title('GP Calibration Curve: Expected vs Observed Coverage', fontsize=14, fontweight='bold')
ax.legend(loc='lower right', fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_xlim([0.48, 1.0])
ax.set_ylim([0.48, 1.0])
ax.set_aspect('equal')
plt.tight_layout()
plt.show()

# Compute calibration error
calibration_error = np.mean(np.abs(observed_coverages - confidence_levels))
print(f"\nMean Absolute Calibration Error: {calibration_error:.4f}")
print(f"Interpretation: Average deviation from perfect calibration")

if calibration_error < 0.05:
    print("Result: EXCELLENT calibration (error < 5%)")
elif calibration_error < 0.10:
    print("Result: GOOD calibration (error < 10%)")
else:
    print("Result: POOR calibration (error >= 10%)")

## 11. Exponential Decay Dataset

Let's test GP on a different nonlinear function: exponential decay.

In [ ]:
# Load exponential decay dataset
dataset_exp = ExponentialDecayDataset(
    n_samples=100,
    noise_model='homoskedastic',
    noise_level=0.05,
    seed=42,
    a=2.0,
    b=3.0,
    c=0.5
)

data_exp = dataset_exp.generate()

print(f"Dataset: {dataset_exp.get_function_form()}")
print(f"Training samples: {len(data_exp.X_train)}")

# Prepare data
X_train_exp = data_exp.X_train.reshape(-1, 1)
y_train_exp = data_exp.y_train
X_test_exp = data_exp.X_test.reshape(-1, 1)
y_test_exp = data_exp.y_test

# Fit GP
kernel_exp = (
    C(1.0, (1e-3, 1e3)) * RBF(0.1, (1e-3, 1.0)) +
    WhiteKernel(0.01, (1e-5, 1e1))
)

gp_exp = GaussianProcessRegressor(
    kernel=kernel_exp,
    n_restarts_optimizer=10,
    alpha=1e-10,
    normalize_y=True,
    random_state=42
)

print("\nFitting GP to exponential decay data...")
gp_exp.fit(X_train_exp, y_train_exp)
print(f"Optimized kernel: {gp_exp.kernel_}")

# Predict
y_pred_exp, y_std_exp = gp_exp.predict(X_plot, return_std=True)

# Evaluate
y_pred_test_exp, y_std_test_exp = gp_exp.predict(X_test_exp, return_std=True)
y_lower_test_exp = y_pred_test_exp - 1.96 * y_std_test_exp
y_upper_test_exp = y_pred_test_exp + 1.96 * y_std_test_exp

rmse_exp = np.sqrt(mean_squared_error(y_test_exp, y_pred_test_exp))
coverage_exp = np.mean((y_test_exp >= y_lower_test_exp) & (y_test_exp <= y_upper_test_exp))
width_exp = np.mean(y_upper_test_exp - y_lower_test_exp)

print(f"\nRMSE: {rmse_exp:.4f}")
print(f"Coverage: {coverage_exp:.3f}")
print(f"Mean Width: {width_exp:.4f}")

In [ ]:
# Plot exponential decay results
y_true_exp = dataset_exp._generate_clean(X_plot.flatten())

fig, ax = plt.subplots(figsize=(14, 6))

# Uncertainty bands
ax.fill_between(X_plot.flatten(), 
                 y_pred_exp - 1.96*y_std_exp, 
                 y_pred_exp + 1.96*y_std_exp,
                 alpha=0.2, color='blue', label='95% CI')
ax.fill_between(X_plot.flatten(), 
                 y_pred_exp - y_std_exp, 
                 y_pred_exp + y_std_exp,
                 alpha=0.4, color='blue', label='68% CI')

# Predictions
ax.plot(X_plot, y_pred_exp, 'b-', linewidth=2.5, label='GP Mean', zorder=5)
ax.plot(X_plot, y_true_exp, 'k--', linewidth=2, label='True Function', zorder=6)

# Training data
ax.scatter(X_train_exp, y_train_exp, c='red', s=70, alpha=0.8, 
           edgecolors='black', linewidth=1, label='Training Data', zorder=10)

ax.set_xlabel('x', fontsize=12)
ax.set_ylabel('y', fontsize=12)
ax.set_title(f'GP on Exponential Decay: y = {dataset_exp.a}*exp(-{dataset_exp.b}*x) + {dataset_exp.c}\n'
             f'Coverage: {coverage_exp:.3f}, RMSE: {rmse_exp:.4f}', 
             fontsize=12, fontweight='bold')
ax.legend(loc='best')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 12. Summary and Key Takeaways

### What We Learned

1. **Gaussian Processes Basics**:
   - GPs are distributions over functions
   - Defined by mean and covariance (kernel) functions
   - Provide closed-form posterior predictions

2. **Kernel Selection**:
   - **RBF**: Very smooth, infinitely differentiable
   - **Matern**: Controlled smoothness via $\nu$ parameter
   - **White Kernel**: Models observation noise
   - Hyperparameters optimized via marginal likelihood

3. **Uncertainty Quantification**:
   - Predictive mean: $\mu(x_*)$
   - Predictive std: $\sigma(x_*)$
   - 95% prediction interval: $[\mu - 1.96\sigma, \mu + 1.96\sigma]$

4. **Interpolation vs Extrapolation**:
   - Low uncertainty near training data (interpolation)
   - High uncertainty far from training data (extrapolation)
   - GP provides honest uncertainty estimates

5. **Calibration**:
   - Well-calibrated GPs: observed coverage ≈ nominal coverage
   - Calibration curves validate prediction intervals

6. **Noise Models**:
   - Standard GP assumes homoskedastic noise
   - For heteroskedastic noise, use input-dependent noise models

### Advantages of GPs

- **Non-parametric**: No fixed functional form
- **Bayesian**: Natural uncertainty quantification
- **Flexible**: Kernel choice encodes prior knowledge
- **Calibrated**: Well-calibrated uncertainty estimates
- **Interpretable**: Clear physical meaning of hyperparameters

### Limitations

- **Computational cost**: $O(n^3)$ scaling for $n$ training points
- **Kernel choice**: Performance depends on kernel selection
- **High dimensions**: Curse of dimensionality (length scale per dimension)
- **Stationarity**: Standard kernels assume stationary covariance

### When to Use GPs

- Small to medium datasets ($n < 10,000$)
- When uncertainty quantification is critical
- Smooth functions with well-defined lengthscales
- When prior knowledge about smoothness is available
- Sequential decision-making (Bayesian optimization)

### Further Reading

- Rasmussen & Williams (2006). *Gaussian Processes for Machine Learning*
- Murphy (2012). *Machine Learning: A Probabilistic Perspective*, Chapter 15
- scikit-learn GP tutorial: https://scikit-learn.org/stable/modules/gaussian_process.html

In [ ]:
# Final summary table
summary_data = {
    'Dataset': ['Gaussian (Homo)', 'Gaussian (Hetero)', 'Exponential Decay'],
    'Kernel': ['RBF + White', 'RBF + White', 'RBF + White'],
    'Coverage': [coverage, gp_noise_results['Homoskedastic']['coverage'], coverage_exp],
    'RMSE': [rmse, np.nan, rmse_exp],
    'Mean Width': [mean_width, np.nan, width_exp]
}

summary_df = pd.DataFrame(summary_data)
print("\n" + "="*60)
print("SUMMARY: GP REGRESSION RESULTS")
print("="*60)
print(summary_df.to_string(index=False))
print("="*60)
print("\nAll models achieve good coverage close to the target 0.95!")
print("This demonstrates the effectiveness of GP for uncertainty quantification.")